In [ ]:
import pandas as pd
import os
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
pd.set_option("display.max_columns", 30)

In [ ]:
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")
url = (
    f"https://data.cityofchicago.org/resource/ajtu-isnz.json?"
    f"$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' "
    f"AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"
)
response = requests.get(url)
data = response.json()


In [ ]:
data

In [ ]:
taxi_trips = pd.DataFrame(data)

In [ ]:
taxi_trips

In [ ]:
taxi_trips.info()

In [ ]:
taxi_trips.describe()

## Transformation: Deal with NaN values

In [ ]:
taxi_trips.drop(["pickup_census_tract", "dropoff_census_tract"], axis=1, inplace=True)
taxi_trips.drop(["pickup_centroid_location", "dropoff_centroid_location"], axis=1, inplace=True)

taxi_trips.dropna(inplace=True)


### Transformation: Renaming

In [ ]:
taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id", 
                           "dropoff_community_area": 
                           "dropoff_community_area_id"},
                  inplace=True)


### Transformation: create helper for the weather table

In [ ]:
taxi_trips["trip_start_timestamp"] = pd.to_datetime(taxi_trips["trip_start_timestamp"])
taxi_trips["datetime_for_weather"] = taxi_trips["trip_start_timestamp"].dt.floor("h")


### Check joining the trips and the weather data 

In [ ]:
## Get the weather data, and create a dataframe out of it.

#extract part

current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime = current_datetime.strftime("%Y-%m-%d")

url = "https://archive-api.open-meteo.com/v1/era5"

params = {
    "latitude": 41.85,
    "longitude": -87.65,
    "start_date": formatted_datetime,
    "end_date": formatted_datetime,
    "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
}
response = requests.get(url, params=params)

weather_data = response.json()

#Transform part
weather_data_filtered = {
    "datetime": weather_data["hourly"]["time"],
    "tempretaure": weather_data["hourly"]["temperature_2m"],
    "wind_speed": weather_data["hourly"]["wind_speed_10m"],
    "rain": weather_data["hourly"]["rain"],
    "precipitation": weather_data["hourly"]["precipitation"],
} 

weather_df = pd.DataFrame(weather_data_filtered)
weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])
weather_df.head()


In [ ]:
taxi_trips_with_weathers = taxi_trips.merge(weather_df, left_on ="datetime_for_weather", right_on = "datetime")

In [ ]:
taxi_trips_with_weathers.head()

### Transformation: Datatype conversions

In [ ]:
data_types = {
    "trip_end_timestamp": "datetime64[ns]",
    "trip_seconds": "int32",
    "trip_miles": "float",
    "pickup_community_area_id": "int8",
    "dropoff_community_area_id": "int8",
    "fare": "float",
    "tips": "float",
    "tolls": "float",
    "extras": "float",
    "trip_total": "float",
}
taxi_trips = taxi_trips.astype(data_types)


In [ ]:
taxi_trips.describe()

##### Memory usage

In [ ]:
## Raw: 18.452.347
## Dtype optimalized: 11.139.659
taxi_trips.memory_usage(deep=True).sum()

#### Sanity checks

In [ ]:
taxi_trips[taxi_trips['trip_end_timestamp'] == taxi_trips['trip_end_timestamp'].max()]


In [ ]:
taxi_trips[taxi_trips['trip_seconds'] == taxi_trips['trip_seconds'].max()]

In [ ]:
taxi_trips[taxi_trips['fare'] == taxi_trips['fare'].max()]

In [ ]:
taxi_trips.nlargest(10, "fare")

In [ ]:
#for i in range(1, len(payment_type_master) + 1):
#    print(i)

In [ ]:
payment_type_master = taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)
payment_type_master = pd.DataFrame(
    {
        "payment_type_id": range(1, len(payment_type_master) + 1),
        "payment_type": payment_type_master
        
    }
    
)

payment_type_master

In [ ]:
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop=True)
company_master = pd.DataFrame(
    {
        "company_id": range(1, len(company_master) + 1),
        "company": company_master
        
    }
    
)

company_master

In [ ]:
taxi_trips[["payment_type","company"]]

In [ ]:
taxi_trips_id = taxi_trips.merge(payment_type_master, on = "payment_type")
taxi_trips_id = taxi_trips_id.merge(company_master, on = "company")

In [ ]:
taxi_trips_id.drop(["payment_type","company"], axis = 1, inplace = True)

In [ ]:
taxi_trips_id.sample(5)

In [ ]:
## Raw: 18.452.347
## Dtype optimalized: 11.139.659
print(taxi_trips.memory_usage(deep=True).sum())
print(taxi_trips_id.memory_usage(deep=True).sum())

In [ ]:
#payment_type_master.to_csv("payment_master_type.csv", index = False)
#company_master.to_csv("company_master.csv", index = False)

### check the modelling

#### payment type

In [ ]:
payment_type_master = taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)
payment_type_master = pd.DataFrame(
    {
        "payment_type_id": range(1, len(payment_type_master) + 1),
        "payment_type": payment_type_master
        
    }
    
)

payment_type_master

In [ ]:
new_payment_type_data = [
{"payment_type": "Credit Card"},
{"payment_type": "x"},
{"payment_type": "y"}
]

new_payment_type_mapping = pd.DataFrame(new_payment_type_data)

new_payment_type_mapping

In [ ]:
payment_type_max_id_value = payment_type_master["payment_type_id"].max()

In [ ]:
range(payment_type_max_id_value + 1,payment_type_max_id_value + len(new_payment_type_mapping) + 1)

In [ ]:
new_payment_type_mapping["payment_type_id"] = range(payment_type_max_id_value + 1,payment_type_max_id_value + len(new_payment_type_mapping) + 1)

In [ ]:
new_payment_type_mapping

In [ ]:
new_payment_types = new_payment_type_mapping[~new_payment_type_mapping["payment_type_id"].isin(payment_type_master["payment_type"])]

In [ ]:
if not new_payment_types.empty:
    payment_type_master = pd.concat([payment_type_master, new_payment_types], ignore_index = True)

In [ ]:
print(payment_type_master)

#### company

In [ ]:
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop=True)
company_master = pd.DataFrame(
    {
        "company_id": range(1, len(company_master) + 1),
        "company": company_master
        
    }
    
)

company_master

In [ ]:
new_company_data = [
{"company": "3556 - 36214 RC Andrews Cab"},
{"company": "x"},
{"company": "y"}
]

new_company_mapping = pd.DataFrame(new_company_data)

new_company_mapping

In [ ]:
company_max_id_value = company_master["company_id"].max()

In [ ]:
new_company_mapping["company_id"] = range(company_max_id_value + 1,company_max_id_value + len(new_company_mapping) + 1)
new_company_mapping

In [ ]:
new_company = new_company_mapping[~new_company_mapping["company_id"].isin(company_master["company"])]

In [ ]:
if not new_company.empty:
    company_master = pd.concat([company_master, new_company], ignore_index = True)

In [ ]:
company_master